In [2]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\PC\Data\ads_fraud_detection


c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


c:/Users/PC/Data/ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [3]:
save_dir=f"{exps_dir}/exp3/exp_gau"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [4]:
test_size=0.33
seed=42
train=pd.read_csv(f'{exps_dir}/exp3/train_EDA.csv')
test=pd.read_csv(f'{exps_dir}/exp3/test_EDA.csv')
train.head()

,isFraud,transactionamt,productcd,card1,card2,card3,card5,card6,addr1,dist1,...,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,P_emaildomain_3,R_emaildomain_1,R_emaildomain_2,R_emaildomain_3
0,0,68.5,W,13926.0,NaN,150.0,142.0,credit,315.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29.0,W,2755.0,404.0,150.0,102.0,credit,325.0,NaN,...,NaN,NaN,NaN,NaN,gmail,com,NaN,NaN,NaN,NaN
2,0,59.0,W,4663.0,490.0,150.0,166.0,debit,330.0,287.0,...,NaN,NaN,NaN,NaN,outlook,com,NaN,NaN,NaN,NaN
3,0,50.0,W,18132.0,567.0,150.0,117.0,debit,476.0,NaN,...,NaN,NaN,NaN,NaN,yahoo,com,NaN,NaN,NaN,NaN
4,0,50.0,H,4497.0,514.0,150.0,102.0,credit,420.0,NaN,...,0.50153,0.487971,0.442759,0.470755,gmail,com,NaN,NaN,NaN,NaN


In [5]:
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]
one_value_cols == one_value_cols_test

True

In [6]:
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]

In [7]:
big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [8]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))
print(cols_to_drop)
cols_to_drop.remove('isFraud')
len(cols_to_drop)

['isFraud', 'c7', 'P_emaildomain_3', 'card3', 'R_emaildomain_3', 'ENTITY_TYPE']


5

In [9]:
train.shape, test.shape

((561013, 98), (29527, 99))

In [10]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
train.head()

,isFraud,transactionamt,productcd,card1,card2,card3,card5,card6,addr1,dist1,...,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,P_emaildomain_3,R_emaildomain_1,R_emaildomain_2,R_emaildomain_3
0,0,68.5,W,13926.0,NaN,150.0,142.0,credit,315.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29.0,W,2755.0,404.0,150.0,102.0,credit,325.0,NaN,...,NaN,NaN,NaN,NaN,gmail,com,NaN,NaN,NaN,NaN
2,0,59.0,W,4663.0,490.0,150.0,166.0,debit,330.0,287.0,...,NaN,NaN,NaN,NaN,outlook,com,NaN,NaN,NaN,NaN
3,0,50.0,W,18132.0,567.0,150.0,117.0,debit,476.0,NaN,...,NaN,NaN,NaN,NaN,yahoo,com,NaN,NaN,NaN,NaN
4,0,50.0,H,4497.0,514.0,150.0,102.0,credit,420.0,NaN,...,0.50153,0.487971,0.442759,0.470755,gmail,com,NaN,NaN,NaN,NaN


In [12]:
train.isnull().sum()

isFraud                 0
transactionamt          0
productcd               0
card1                   0
card2                8604
                    ...  
P_emaildomain_2     89737
P_emaildomain_3    558111
R_emaildomain_1    429350
R_emaildomain_2    429443
R_emaildomain_3    558436
Length: 98, dtype: int64

In [13]:
continuous_cols = train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns
categorical_cols

Index(['productcd', 'card6', 'p_emaildomain', 'r_emaildomain', 'devicetype',
       'deviceinfo', 'EVENT_ID', 'ENTITY_ID', 'EVENT_TIMESTAMP', 'ENTITY_TYPE',
       'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3',
       'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3'],
      dtype='object')

In [14]:
continuous_cols = train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns

for col in continuous_cols:
    train[col].interpolate(method='linear', inplace=True)

for col in categorical_cols:
    train[col].fillna(train[col].mode()[0], inplace=True)

train.isnull().sum()

isFraud            0
transactionamt     0
productcd          0
card1              0
card2              1
                  ..
P_emaildomain_2    0
P_emaildomain_3    0
R_emaildomain_1    0
R_emaildomain_2    0
R_emaildomain_3    0
Length: 98, dtype: int64

In [15]:
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [17]:
train.columns

Index(['isFraud', 'transactionamt', 'productcd', 'card1', 'card2', 'card5',
       'card6', 'addr1', 'dist1', 'p_emaildomain', 'r_emaildomain', 'c1', 'c2',
       'c4', 'c5', 'c6', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'v62',
       'v70', 'v76', 'v78', 'v82', 'v91', 'v127', 'v130', 'v139', 'v160',
       'v165', 'v187', 'v203', 'v207', 'v209', 'v210', 'v221', 'v234', 'v257',
       'v258', 'v261', 'v264', 'v266', 'v267', 'v271', 'v274', 'v277', 'v283',
       'v285', 'v289', 'v291', 'v294', 'id_01', 'id_02', 'id_05', 'id_06',
       'id_09', 'id_13', 'id_17', 'id_19', 'id_20', 'devicetype', 'deviceinfo',
       'EVENT_ID', 'ENTITY_ID', 'EVENT_TIMESTAMP',
       'transactionamt_to_mean_card1', 'transactionamt_to_mean_card2',
       'transactionamt_to_std_card1', 'transactionamt_to_mean_card3',
       'transactionamt_to_std_card3', 'transactionamt_to_mean_card5',
       'transactionamt_to_std_card5', 'transactionamt_to_mean_card6',
       'transactionamt_to_std_card6', 'transac

In [18]:
X_train = train.drop(['isFraud'], axis=1)
y_train = train['isFraud']


# del train
# test = test[["TransactionDT", 'TransactionID']]

In [19]:
y_train

0         0
1         0
2         0
3         0
4         0
         ..
561008    0
561009    0
561010    0
561011    0
561012    0
Name: isFraud, Length: 561013, dtype: int64

In [20]:
y_train.values

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.5179993684432125, 1: 14.389376218323587}

In [22]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_rosampled, y_rosampled = ros.fit_resample(train.drop(columns=['isFraud']), train['isFraud'])
X_rosampled = pd.DataFrame(X_rosampled)
y_rosampled = pd.DataFrame(y_rosampled)
oversampling_sample=pd.concat([X_rosampled, y_rosampled], axis=1)    

In [23]:
from sdv.metadata import SingleTableMetadata
metadata_sync = SingleTableMetadata()
metadata_sync.detect_from_dataframe(data=oversampling_sample)

In [24]:
from sdv.single_table import GaussianCopulaSynthesizer



gau_synthesizer = GaussianCopulaSynthesizer(metadata_sync,
                                            enforce_min_max_values=True,  
                                            enforce_rounding=True,
                                            default_distribution='gamma') 

gau_synthesizer.fit(oversampling_sample)
gau_synthesizer.save(
    filepath=f'{save_dir}/gau_synthesizer.pkl'
)

c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\sdv\single_table\base.py:120: FutureWarning:

The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.

c:\Users\PC\Data\ads_fraud_detection\.conda\lib\site-packages\sdv\single_table\base.py:105: UserWarning:

We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.



In [25]:
gau_synthetic_data = gau_synthesizer.sample(num_rows=len(train))
gau_synthetic_data.head()

,transactionamt,productcd,card1,card2,card5,card6,addr1,dist1,p_emaildomain,r_emaildomain,...,id_02_to_std_card3,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,R_emaildomain_1,R_emaildomain_2,isFraud
0,0.251,C,10379,1567,793,2,2456,0.937947,ustewart@example.net,lynn17@example.net,...,2.658321,2.245877,0.000566,2.222846,2.425723,moodykathleen@example.org,angelabell@example.net,david62@example.net,uwright@example.org,1
1,0.251,W,13552,408,596,1,0,620.035746,mbowen@example.net,jessica66@example.com,...,1.001658,0.830985,0.000011,0.864802,0.935059,pgamble@example.org,david54@example.org,burchcrystal@example.net,matthewmendoza@example.org,0
2,0.251,R,2723,1122,486,2,2567,0.580808,iwalker@example.org,joel03@example.org,...,0.791038,0.774131,0.000011,0.605593,0.671258,wallsjoseph@example.org,douglas76@example.com,terrance46@example.com,lfrye@example.net,0
3,0.251,C,3373,1530,762,1,1293,0.308398,scottmcdaniel@example.com,rodgerssheryl@example.net,...,1.709565,1.533707,0.000011,1.460910,1.603135,harold46@example.com,craig56@example.com,laura56@example.net,xrivas@example.org,0
4,0.251,W,5498,638,355,2,283,238.289087,morgan74@example.net,davisjoseph@example.com,...,0.864057,0.831410,0.000011,0.813342,0.832720,carlsonsteven@example.net,sjimenez@example.net,whitekari@example.com,eileengreen@example.org,1


In [26]:
from sdv.evaluation.single_table import run_diagnostic



diagnostic_gau = run_diagnostic(
    real_data=oversampling_sample,
    synthetic_data=gau_synthetic_data,
    metadata=metadata_sync
)




Generating report ...

(1/2) Evaluating Data Validity: |██████████| 93/93 [00:03<00:00, 27.28it/s]|
Data Validity Score: 99.95%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 333.28it/s]|
Data Structure Score: 91.75%

Overall Score (Average): 95.85%



In [27]:
from sdv.evaluation.single_table import evaluate_quality



diagnostic_gau = evaluate_quality(
    real_data=oversampling_sample,
    synthetic_data=gau_synthetic_data,
    metadata=metadata_sync
)




Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 93/93 [06:51<00:00,  4.42s/it]|
Column Shapes Score: 55.9%

(2/2) Evaluating Column Pair Trends: |██████████| 4278/4278 [09:10<00:00,  7.77it/s]|
Column Pair Trends Score: 95.48%

Overall Score (Average): 75.69%



In [28]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

## Gau data
gau_data=gau_synthetic_data[gau_synthetic_data.isFraud==1]
gau_sample=pd.concat([train, gau_data] , axis=0, ignore_index=True)
X_gausampled, y_gausampled = rus.fit_resample(gau_sample.drop(columns=['isFraud']), gau_sample['isFraud'])
X_gausampled = pd.DataFrame(X_gausampled)
y_gausampled = pd.DataFrame(y_gausampled)
gau_sample=pd.concat([X_gausampled, y_gausampled], axis=1)    
gau_sample = shuffle(gau_sample, random_state=42)
print(gau_sample.isFraud.value_counts())

isFraud
1    299835
0    299835
Name: count, dtype: int64


In [ ]:
cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']
for col in cat_cols:
    if col in gau_sample.columns:
        le = LabelEncoder()
        le.fit(list(gau_sample[col].astype(str).values) + list(test[col].astype(str).values))
        gau_sample[col] = le.transform(list(gau_sample[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))   

In [29]:
X_train = gau_sample.drop(columns=['isFraud'])
y_train = gau_sample['isFraud']

In [30]:
from sklearn.model_selection import train_test_split
# x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.20, random_state = 101)
X_train.to_csv(f'{save_dir}/x_train.csv', index=False)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)
# x_valid.to_csv(f'{save_dir}/x_valid.csv', index=False)
# y_valid.to_csv(f'{save_dir}/y_valid.csv', index=False)

In [31]:

for col in continuous_cols:
    test[col].interpolate(method='linear', inplace=True)

for col in categorical_cols:
    test[col].fillna(test[col].mode()[0], inplace=True)

x_test = test.drop(['isFraud'], axis=1)
y_test = test['isFraud']

x_test.to_csv(f'{save_dir}/x_test.csv', index=False)
y_test.to_csv(f'{save_dir}/y_test.csv', index=False)

KeyError: 'card3'